# Downloading the data from earthscope

In [2]:
import requests
from pathlib import Path
import hatanaka
from earthscope_sdk.auth.device_code_flow import DeviceCodeFlowSimple
from earthscope_sdk.auth.auth_flow import NoTokensError
import os

## earthscope authentication
You will need to create an account and get a token fron the earthscope website to download data.
Information can be found here: https://www.unavco.org/data/gps-gnss/data-access-methods/data-access-methods.html#searchdata

In [ ]:
# choose where you want the token saved - the default file name is sso_tokens.json
token_path = "./"

In [3]:
def download_data(token_path, year, dayOfYear, station, path_out):
    url = f"https://gage-data.earthscope.org/archive/gnss/rinex/obs/{year}/{dayOfYear}/{station}{dayOfYear}0.{str(year)[2:]}d.Z"
    print(url)
    # create station folder (if it doesn't already exist)
    station_path = f"{path_out}{year}/{station}/"
    Path(station_path).mkdir(parents=True, exist_ok=True)
    device_flow = DeviceCodeFlowSimple(Path(token_path))
    try:
        # get access token from local path
        device_flow.get_access_token_refresh_if_necessary()
    except NoTokensError:
        # if no token was found locally, do the device code flow
        device_flow.do_flow()
    token = device_flow.access_token

    # request a file and provide the token in the Authorization header
    file_name = Path(url).name
    directory_to_save_file = Path.cwd() # where you want to save the downloaded file 

    r = requests.get(url, headers={"authorization": f"Bearer {token}"})
    if r.status_code == requests.codes.ok:
        with open(station_path + file_name, 'wb') as f:
            f.write(r.content)
        hatanaka.decompress_on_disk(station_path + file_name)
        os.remove(station_path + file_name)

    else:
        #problem occured
        print(f"failure: {r.status_code}, {r.reason}")

## Call the download function using the following arguments:
* Token path (unavco token)
* Year to download
* day of the year to download: Format needs to be a sting with 3 char, for example the 3rd day of the year will be "003"
* Station
* path to folder where to save the file

In [5]:
path_out = './raw_rinex/'

for i, station in enumerate(['albh']):
    for day in range(1, 2):
        download_data(token_path, 2020, str(day).zfill(3), station, path_out)

https://gage-data.earthscope.org/archive/gnss/rinex/obs/2020/001/albh0010.20d.Z
